<a href="https://colab.research.google.com/github/mariasavu/CNN-Covid-19vsPulmonaryCancerClassifcation-/blob/main/lastChance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

!echo "Loading data"
#!pip install gdown
#trebuie verficat path ul
!unzip '/content/drive/MyDrive/date_80_20.zip'
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras
import matplotlib.pyplot as plt
from tensorflow.keras import applications
from tensorflow.keras import models, layers
from tensorflow.keras import optimizers
import os
import shutil
import tensorflow as tf
import pickle
from tensorflow.keras.applications import ResNet50V2

def save_model(model, history, name):
    model.save(f'saved_models/{name}.h5')

    with open(f'saved_models/{name}_history.pickle', mode='wb') as f:
        pickle.dump(history.history, f)
def plot_history(h):
    fig, axes = plt.subplots(1, 2, figsize=(20, 5))
    ax = axes[0]
    ax.plot(h.history['acc'], 'r', label='Training acc')
    ax.plot(h.history['val_acc'], 'g', label='Validation acc')
    ax.set_title('Training and validation accuracy')
    ax.legend()
    ax.grid()

    ax = axes[1]
    ax.plot(h.history['loss'], 'r', label='Training loss')
    ax.plot(h.history['val_loss'], 'g', label='Validation loss')
    ax.set_title('Training and validation loss')
    ax.legend()

In [ ]:
root_path = '/content/date_80_20'
train_dir =os.path.join(root_path, 'Train')
test_dir = os.path.join(root_path, 'Test')

train_mal_dir = os.path.join(train_dir, 'Cancer')
train_covid_dir = os.path.join(train_dir, 'Covid')

test_mal_dir = os.path.join(test_dir, 'Cancer')
test_covid_dir = os.path.join(test_dir, 'Covid')


print('training malign images:', len(os.listdir(train_mal_dir)))
print('training covid images:', len(os.listdir(train_covid_dir)))
print('testing malign images:', len(os.listdir(test_mal_dir)))
print('testing covid images:', len(os.listdir(test_covid_dir)))

training malign images: 451
training covid images: 440
testing malign images: 106
testing covid images: 107


In [ ]:
#function to remove the files added by MacOS while adding files to archive as it may be interpreted as a diffrent class in our network
def remove_ds_store(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file == '.DS_Store':
                file_path = os.path.join(root, file)
                os.remove(file_path)


# Replace with the path to your directory
remove_ds_store(train_dir)
remove_ds_store(test_dir)



In [ ]:
#checking to see if the class names are correct
class_names = os.listdir(test_dir)
num_classes = len(class_names)

print("Number of classes:", num_classes)
print("Class names:", class_names)

Number of classes: 2
Class names: ['Cancer', 'Covid']


In [ ]:
#starting with data augumentation
import numpy as np
import random
BATCH_SIZE = 64
size = (120, 120)
seed = random.randint(0, 9999)

valid_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Create a flow from the directory for validation data - seed=42
valid_gen = valid_datagen.flow_from_directory(train_dir, subset='validation',
                                              shuffle=True, seed=seed,
                                              target_size=size,
                                              batch_size=BATCH_SIZE)
print(f'valid_samples: {valid_gen.n}')


train_datagen = ImageDataGenerator(
    rescale=1./255,

    rotation_range=30,  # Increased rotation range
    width_shift_range=0.2,  # Increased shift range
    height_shift_range=0.2,  # Increased shift range
    validation_split=0.2,
    zoom_range=0.2,  # Increased zoom range
    horizontal_flip=True,
    vertical_flip=True,  # Added vertical flip
    shear_range=0.2,  # Added shear range
    fill_mode='nearest',
    brightness_range=(0.8, 1.2),  # Adjust brightness
    channel_shift_range=20,  # Random channel shifts
    # Add more augmentation techniques as desired
)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    subset="training",
    shuffle=True,
    seed=seed,
    target_size=size,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
train_samples = train_generator.n
print(f'train_samples: {train_samples}')
test_datagen = ImageDataGenerator(preprocessing_function=applications.vgg19.preprocess_input)


test_generator = test_datagen.flow_from_directory(
    test_dir,
    shuffle=False,
    seed=seed,
    target_size=size,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
test_samples = test_generator.n
print(f'test_samples: {test_samples}')





total_train_images = train_samples * BATCH_SIZE
print("Total number of images during training:", total_train_images)




Found 177 images belonging to 2 classes.
valid_samples: 177
Found 712 images belonging to 2 classes.
train_samples: 712
Found 211 images belonging to 2 classes.
test_samples: 211
Total number of images during training: 45568


In [ ]:
#def the model arhitecture using the ResNet base
checkpoint_filepath = '/checkpoint'
MODEL_FNAME='ResNet-80-20V2'
epoci = 50
conv_base = applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=size + (3,)
)
conv_base.trainable = False
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.02)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu', kernel_regularizer='l2'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(2, activation='sigmoid'))
model.summary()

model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=0.0001),
    metrics=['acc']
)
early_stop=tf.keras.callbacks.EarlyStopping( monitor="val_loss", patience=4, verbose=1, restore_best_weights=True)
rlronp=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.4,  patience=1, verbose=1)
weight_m = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
hist = model.fit(
    train_generator,
    validation_data=valid_gen,
    epochs=epoci,
    verbose=1,
    callbacks=[early_stop, rlronp],
    validation_steps=None,  shuffle=True
)
performance=model.evaluate( test_generator,  steps=BATCH_SIZE, verbose=1 )[1] * 100
save_model(model, hist, MODEL_FNAME) #    ax.grid()
train_loss=min(hist.history['loss'])
val_loss=min(hist.history['val_loss'])
train_acc=max(hist.history['acc'])
val_acc=max(hist.history['val_acc'])
plot_history(hist)
print('train_loss',train_loss)
print('val_loss',val_loss)
print('train_acc',train_acc*100)
print('val_acc',val_acc*100)
print('test_acc ', performance, ' %')

In [ ]:
# creating a function that would plot the training process
def plot_history(h):
    fig, axes = plt.subplots(1, 2, figsize=(20, 5))
    ax = axes[0]
    ax.plot(h.history['accuracy'], 'r', label='Training accuracy')
    ax.plot(h.history['val_accuracy'], 'g', label='Validation accuracy')
    ax.set_title('Training and validation accuracy')
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Accuracy')
    ax.legend()

    ax = axes[1]
    ax.plot(h.history['loss'], 'r', label='Training loss')
    ax.plot(h.history['val_loss'], 'g', label='Validation loss')
    ax.set_title('Training and validation loss')
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Loss')
    ax.legend()

    plt.show()

In [ ]:
from keras.applications.vgg19 import VGG19
model =VGG19(weights = "imagenet", include_top=False, input_shape = size + (3,))
#model.layers

80134624/80134624 [==============================] - 1s 0us/step


In [ ]:
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.models import Model

for layer in model.layers:
    layer.trainable = False

#Adding custom Layers
x = model.output

x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)

model = Model(inputs=model.input, outputs=predictions)
y_pred = model.predict(X_test)
confusion_matrix = sklearn.metrics.confusion_matrix(y_test, np.rint(y_pred))
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 120, 120, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 120, 120, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 120, 120, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 60, 60, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 60, 60, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 60, 60, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 30, 30, 128)       0     

In [ ]:
#compile the model
opt =optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

early_stop=tf.keras.callbacks.EarlyStopping( monitor="val_loss", patience=4, verbose=1, restore_best_weights=True)
rlronp=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.4,  patience=1, verbose=1)
weight_m = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
hist = model.fit(
    train_generator,
    validation_data=valid_gen,
    epochs=epoci,
    verbose=1,
    callbacks=[early_stop, rlronp],
    validation_steps=None,  shuffle=True
)
performance=model.evaluate( test_generator,  steps=BATCH_SIZE, verbose=1 )[1] * 100
save_model(model, hist, MODEL_FNAME) #    ax.grid()
train_loss=min(hist.history['loss'])
val_loss=min(hist.history['val_loss'])
train_acc=max(hist.history['accuracy'])
val_acc=max(hist.history['val_acc'])
plot_history(hist)
print('train_loss',train_loss)
print('val_loss',val_loss)
print('train_acc',train_acc*100)
print('val_acc',val_acc*100)
print('test_acc ', performance, ' %')

In [ ]:
from keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.models import Model
from tensorflow.keras.layers import BatchNormalization
checkpoint_filepath = '/checkpoint'
MODEL_FNAME='VGG-80-20VBATCH64SGD' # 4 are mai multe straturi
epoci = 80
model = VGG19(weights="imagenet", include_top=False, input_shape=size + (3,))
for layer in model.layers:
    layer.trainable = False


x = model.output
x = Flatten()(x)
x = Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.03))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation="sigmoid")(x)

model = Model(inputs=model.input, outputs=predictions)

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 120, 120, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 120, 120, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 120, 120, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 60, 60, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 60, 60, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 60, 60, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 30, 30, 128)       0   

In [ ]:
#training the model with the VGG-19 base
opt = optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=4, verbose=1, restore_best_weights=True)
rlronp = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.4, patience=1, verbose=1)
weight_m = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', mode='max', save_best_only=True)

hist = model.fit(
    train_generator,
    validation_data=valid_gen,
    epochs=epoci,
    verbose=1,
    callbacks=[early_stop, rlronp, weight_m],
    validation_steps=len(valid_gen),
    shuffle=True
)

# Evaluate on the test set
steps = test_generator.samples // BATCH_SIZE
performance = model.evaluate(test_generator, steps=steps, verbose=1)[1] * 100

# Save the model and its history
model.save(MODEL_FNAME + '.h5')
with open(MODEL_FNAME + '_history.pkl', 'wb') as file:
    pickle.dump(hist.history, file)

# Print and display results
train_loss = min(hist.history['loss'])
val_loss = min(hist.history['val_loss'])
train_acc = max(hist.history['accuracy'])
val_acc = max(hist.history['val_accuracy'])
plot_history(hist)

print('train_loss:', train_loss)
print('val_loss:', val_loss)
print('train_acc:', train_acc * 100)
print('val_acc:', val_acc * 100)
print('test_acc:', performance, '%')


In [ ]:
# to store the modeles directly to the drive

!cp -r "/content/saved_models" "/content/drive/MyDrive"

In [ ]:
!cp -r "/content/VGG-80-20VBATCH64_history.pkl" "/content/drive/MyDrive"